In [12]:
import time
import urllib.parse
import hashlib
import hmac
import base64
import os
import requests

In [13]:
def current_milli_time():
    return str(int(1000*time.time()))

In [14]:
def get_kraken_signature(urlpath, data, secret):

    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()

In [15]:
# Attaches auth headers and returns results of a POST request
def kraken_request(uri_path, data, api_key, api_sec):
    headers = {}
    headers['API-Key'] = api_key
    # get_kraken_signature() as defined in the 'Authentication' section
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

In [16]:
def buy_order_best_price(api_key, api_sec, wish_coin_price, usd_vol = 10):
    uri_path = '/0/private/AddOrder'
    coin_vol = round((usd_vol/wish_coin_price),5)
    data = {
        "nonce": current_milli_time(),
        "ordertype": "market", 
        "pair": "XBTUSD",
        "type": "buy",
        "volume": coin_vol,
        
    }
    
    try:
        resp = kraken_request(uri_path, data, api_key, api_sec).json()
        print(f"order: {resp['result']['descr']['order']}\ttxid: {resp['result']['txid']}\terrors: {resp['error']}")
    except KeyError as e:
        print(resp)
        raise(e)
    return {'txid': resp['result']['txid'][0], 'coin_vol': float(resp['result']['descr']['order'].split(' ')[1])}

    print(data)
    
api_url = 'https://api.kraken.com'
api_sec = 'OPMkSRkt9lgLE5RRsWD7VKnstU8L4mEsr/K45wV9wOycmutTTod/D13/owSadiqB6mm6CMdOqFL7KnFuIrRFmw=='
api_key = 'MQN3A/ARCcCVP0TTIladpbcrVTwvdRDHLNJ05p5wN2tnBhz4DGmXDCIL'


buy_order_best_price(api_key, api_sec, 49020.6, 10)


{'error': [], 'result': {'descr': {'order': 'buy 0.00020000 XBTUSD @ market'}, 'txid': ['OXTRJM-NCX6G-3LZ3UL']}}
{'nonce': '1629956023827', 'ordertype': 'market', 'pair': 'XBTUSD', 'type': 'buy', 'volume': 0.0002}


In [ ]:
def get_order_info(api_key, api_sec, order):
    uri_path = '/0/private/QueryOrders'
    data = {
        "nonce": current_milli_time(),
        "txid": order['txid'],
        "trades": False
    }
    
    try:
        resp = kraken_request(uri_path, data, api_key, api_sec).json()
        print(f"chekin info for order {order}")
    except KeyError as e:
        print(resp)
        raise(e)
        
    order_info = {
        'txid'   : order['txid'],
        'pair'   : resp['result'][order['txid']]['descr']['pair'],
        'type'   : resp['result'][order['txid']]['descr']['type'],
        'status' : resp['result'][order['txid']]['status'], # open, closed, canceled 
        'price'  : float(resp['result'][order['txid']]['price']),
        'cost'   : float(resp['result'][order['txid']]['cost']),
        'vol'    : float(resp['result'][order['txid']]['vol']),
    }
    return order_info

# api_url = 'https://api.kraken.com'
# api_sec = 'OPMkSRkt9lgLE5RRsWD7VKnstU8L4mEsr/K45wV9wOycmutTTod/D13/owSadiqB6mm6CMdOqFL7KnFuIrRFmw=='
# api_key = 'MQN3A/ARCcCVP0TTIladpbcrVTwvdRDHLNJ05p5wN2tnBhz4DGmXDCIL'


# order = {'txid': 'OXTRJM-NCX6G-3LZ3UL', 'coin_vol': 1}
# order_info = get_order_price_vol(api_key, api_sec, order)

# print(order_info)


In [17]:
def buy_order(api_key, api_sec, coin_price, usd_vol = 10):
    uri_path = '/0/private/AddOrder'
    coin_vol = round((10/coin_price),5)
    print(coin_vol)
    data = {
        "nonce": current_milli_time(),
        "ordertype": "limit", 
        "pair": "XBTUSD",
        "price": coin_price, 
        "type": "buy",
        "volume": coin_vol
    }
    try:
        resp = kraken_request(uri_path, data, api_key, api_sec).json()
        print(f"order: {resp['result']['descr']['order']}\ttxid: {resp['result']['txid']}\terrors: {resp['error']}")
    except KeyError as e:
        print(resp)
        raise(e)
    return {'txid': resp['result']['txid'][0], 'coin_vol': float(resp['result']['descr']['order'].split(' ')[1])}

In [7]:
def sell_order(api_key, api_sec, coin_price, coins_vol):
    uri_path = '/0/private/AddOrder'
    data = {
        "nonce": current_milli_time(),
        "ordertype": "limit", 
        "pair": "XBTUSD",
        "price": coin_price, 
        "type": "sell",
        "volume": coins_vol
    }
    try:
        resp = kraken_request(uri_path, data, api_key, api_sec).json()
        print(f"order: {resp['result']['descr']['order']}\ttxid: {resp['result']['txid']}\terrors: {resp['error']}")
    except KeyError as e:
        print(resp)
        raise(e)
    return {'txid': resp['result']['txid'][0], 'usd_vol': coin_price*coins_vol}

In [8]:
def check_order(api_key, api_sec, order):
    uri_path = '/0/private/QueryOrders'
    data = {
        "nonce": current_milli_time(),
        "txid": order['txid'],
        "trades": False
    }
    
    try:
        resp = kraken_request(uri_path, data, api_key, api_sec).json()
        print(f"status txid: {order['txid']}\tstatus: {resp['result'][order['txid']]['status']}\terrors: {resp['error']}")
    except KeyError as e:
        print(resp)
        raise(e)
    # open, closed, canceled 
    return resp['result'][order['txid']]['status']

In [9]:
def cancel_order(api_key, api_sec, order):
    uri_path = '/0/private/CancelOrder'
    data = {
        "nonce": current_milli_time(),
        "txid": order['txid']
    }
    
    try:
        resp = kraken_request(uri_path, data, api_key, api_sec).json()
        print(f"cancel txid: {order['txid']}\terrors: {resp['error']}")
    except KeyError as e:
        print(resp)
        raise(e)
        
    return order['txid']

In [11]:
%%time
api_url = 'https://api.kraken.com'
api_sec = 'OPMkSRkt9lgLE5RRsWD7VKnstU8L4mEsr/K45wV9wOycmutTTod/D13/owSadiqB6mm6CMdOqFL7KnFuIrRFmw=='
api_key = 'MQN3A/ARCcCVP0TTIladpbcrVTwvdRDHLNJ05p5wN2tnBhz4DGmXDCIL'

order = buy_order(api_key, api_sec, 43000, usd_vol = 10)
canceled_txid = cancel_order(api_key, api_sec, order)

order = sell_order(api_key, api_sec, 70000, order['coin_vol'])
canceled_txid = cancel_order(api_key, api_sec, order)

order_status = check_order(api_key, api_sec, order)



0.00023
order: buy 0.00023000 XBTUSD @ limit 43000.0	txid: ['O4VMES-D4KFR-EJXGWU']	errors: []
cancel txid: O4VMES-D4KFR-EJXGWU	errors: []
order: sell 0.00023000 XBTUSD @ limit 70000.0	txid: ['OKZP6M-4ZFNH-UPEBDY']	errors: []
cancel txid: OKZP6M-4ZFNH-UPEBDY	errors: []
status txid: OKZP6M-4ZFNH-UPEBDY	status: canceled	errors: []
Wall time: 4.26 s
